# Tutorial for performing eQTL analysis for a dataset with many individuals.

Most of this tutorial can also be used for testing any feature that is at the replicate/individual level. For example, comparing case vs control would use similar procedure, since the independent variable is defined for each person and not for each cell.

To install `memento` in the pre-release version (for Ye Lab members), install it directly from github by running:

```pip install git+https://github.com/yelabucsf/scrna-parameter-estimation.git@release-v0.0.8```

This requires that you have access to the Ye Lab organization. Be sure to delete any old versions of `memento` via `pip uninstall memento` if you played with the method before.

In [1]:
import scanpy as sc
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import itertools
from pybedtools import BedTool
import statsmodels.formula.api as smf
import statsmodels.api as sm
import imp

import os
import pickle as pkl
%matplotlib inline

In [2]:
import sys
# sys.path.append('/home/ssm-user/Github/scrna-parameter-estimation/dist/memento-0.0.8-py3.8.egg')
import memento

In [3]:
data_path  = '/data_volume/memento/lupus/'

### Read the inputs: variables of interest (SNPs), covariates, SNP-gene pairs.

For each of these SNP and covariate, each row is an individual and columns are different variables of interest. 

For the tutorial, we use the genotypes and covariates used in 2022 Perez, Gordon, Subramaniam et al. paper from the lab. These inputs are identical to Matrix eQTL inputs - I just transpose them here because I think it makes more sense that observations are rows...

For the tutorial, we just setup some random SNP-gene pairs to test; however, you can flexibly design this mapping to fit your needs. I purposefully didn't encode all possible variations of how you can define gene-SNP relationships.

In [4]:
pop = 'eur'
snps_path = data_path + 'mateqtl_input/{}_genos.tsv'.format(pop)
cov_path = data_path + 'mateqtl_input/{}_mateqtl_cov.txt'.format(pop)

In [5]:
snps = pd.read_csv(snps_path, sep='\t', index_col=0).T
cov = pd.read_csv(cov_path, sep='\t', index_col=0).T

In [6]:
# Print the first 5 SNPs for the first 5 individuals to show the structure
snps.iloc[:, :5].head(5)

CHROM:POS,3:165182446,6:122682327,22:40561759,3:104381193,15:57107863
1132_1132,1,1,1,1,2
1285_1285,2,1,2,1,2
1961_1961,1,2,2,0,2
HC-526,1,1,2,2,1
1414_1414,0,0,2,1,2


In [7]:
# Print the covariates for the first 5 individuals to show the structure
cov.head(5)

,age,Female,status,PC1_e,PC2_e,PC3_e,PC4_e,PC5_e,PC6_e,PC7_e,...,batch_cov_b_14,batch_cov_b_15,batch_cov_b_2,batch_cov_b_3,batch_cov_b_4,batch_cov_b_5,batch_cov_b_6,batch_cov_b_7,batch_cov_b_8,batch_cov_b_9
1132_1132,45.0,0.0,0.0,19.067178,17.787198,10.275343,-2.829570,-3.546597,-1.269196,-2.183796,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1285_1285,39.0,0.0,0.0,14.471841,18.737343,12.465061,11.195105,-2.246129,-11.168822,2.230269,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1961_1961,43.0,0.0,0.0,-7.343628,38.241007,-7.431836,0.607220,-13.730105,-2.339229,-3.238375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
HC-526,58.0,0.0,1.0,0.495487,-17.795535,0.458286,5.384761,-10.269823,-2.239953,-4.240055,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1414_1414,41.0,1.0,0.0,-10.313840,-3.423322,1.635042,9.192646,-3.507571,11.446228,-3.834848,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [8]:
# You can define this mapping DataFrame however you want - for example, you can take find gene-SNP pairs via looking for a 100kb.
# Here, to make the tutorial faster, we'll just randomly take 50k lines.
gene_snp_pairs = pd.read_csv(data_path + 'mateqtl_input/{}/gene_snp_pairs_hg19_100kb.csv'.format(pop))
gene_snp_pairs.columns = ['gene', 'SNP']
gene_snp_pairs = gene_snp_pairs.query('SNP in @snps.columns').sample(50000)

In [10]:
gene_snp_pairs.head(10)

,gene,SNP
5458273,PEX3,6:143704295
4909714,SLC30A5,5:68296445
3722035,BOK,2:242503543
3540175,AMER3,2:131610074
3619623,MYO1B,2:192016879
816583,VAX1,10:118985613
5928785,RP1,8:55598202
493305,C1orf116,1:207165730
2422224,ZMYND15,17:4690500
583777,OR2T33,1:248481384


### Read h5ad object

Standard h5ad file in the scanpy workflow. Some things to keep in mind:

- `adata.X` should be the raw counts with all genes detected. Typically, this will be the size of N cells with ~30k genes in a standard 10X experiment. 
- Here, we will just use the T4 cells defined by one of the AnnData.obs columns.


In [11]:
ct = 'T4'

In [12]:
adata = sc.read(data_path + 'single_cell/{}_{}.h5ad'.format(pop, ct))
adata = adata[adata.obs.ind_cov.isin(snps.index)].copy() # pick out individuals we have genotype and covariates for


In [13]:
print('We have {} cells labeled as T4'.format(adata.shape[0]))

We have 129531 cells labeled as T4


In [14]:
adata.obs.head(3)

,batch_cov,ind_cov,Processing_Cohort,louvain,cg_cov,ct_cov,L3,ind_cov_batch_cov,Age,Sex,pop_cov,Status,SLE_status
GTCACGGAGATTACCC-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0-0-0-0-0-0-0-0-0,dmx_YE_8-2,1368_1368,2.0,1,T4,NaN,0.0,1368_1368:dmx_YE_8-2,45.0,Male,European,Managed,SLE
GTCATTTCAGAGTGTG-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0-0,dmx_YS-JY-22_pool5,HC-540,4.0,2,T4,T4_em,1.0,HC-540:dmx_YS-JY-22_pool5,68.0,Female,European,Healthy,Healthy
AAAGATGGTTCACGGC-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0-0-0-0-0-0,dmx_YS-JY-20_pool3,HC-006,4.0,1,T4,T4_naive,1.0,HC-006:dmx_YS-JY-20_pool3,53.0,Female,European,Healthy,Healthy


In [15]:
adata.X[:5, :15]


<5x15 sparse matrix of type '<class 'numpy.float32'>'
	with 0 stored elements in Compressed Sparse Row format>

In [16]:
# adata.X should be a sparse matrix with counts
print('Confirming that adata.X is a sparse matrix of counts.')
print('Row sums are:')
print(adata.X.sum(axis=1)[:5])
print('')
print('The matrix itself:')
adata.X

Confirming that adata.X is a sparse matrix of counts.
Row sums are:
[[1905.]
 [2104.]
 [2102.]
 [1209.]
 [2030.]]

The matrix itself:


<129531x32738 sparse matrix of type '<class 'numpy.float32'>'
	with 83322139 stored elements in Compressed Sparse Row format>

### Define the helper function for running `memento` on genetics data

Next iteration maybe this will be a function in the package. For now, define the function manually.

In [17]:
def run_genetics(adata, snps, cov, gene_snp_pairs, num_cpu, num_blocks=5, num_boot=5000):
    """
    Run mean eQTL analysis using memento. adata, snp, cov, gene_snp_pairs should be specified in the format above.
    """
    
    # Setup memento
    adata.obs['capture_rate'] = 0.1
    memento.setup_memento(adata, q_column ='capture_rate', trim_percent=0.1, filter_mean_thresh=0.05, estimator_type='mean_only')
    memento.create_groups(adata, label_columns=['ind_cov'])
    
    # To prevent memory issues, we will separate out some SNPs for each gene
    reordered_gene_snp_pairs = gene_snp_pairs.sample(frac=1)
    
    # Re-order the SNP and covariate DataFrames to the order that memento expects
    sample_order = memento.get_groups(adata) #the order of samples that memento expects
    cov_df = cov.loc[sample_order['ind_cov']]
    snps_df = snps.loc[sample_order['ind_cov']]
    
    # Number of tests in each block
    blocksize=int(gene_snp_pairs.shape[0]/num_blocks)
    print('blocksize', blocksize)
    
    # Run memento for each block
    results = []
    for block in range(num_blocks):
        print('working on block', block)
        subset_gene_snp_pairs = reordered_gene_snp_pairs.iloc[(blocksize*block):(blocksize*(block+1)), :]

        memento.compute_1d_moments(adata, min_perc_group=.5, gene_list=subset_gene_snp_pairs.gene.drop_duplicates().tolist())

        gene_to_snp_dictionary = dict(subset_gene_snp_pairs[subset_gene_snp_pairs.gene.isin(adata.var.index)].groupby('gene')['SNP'].apply(list))

        memento.ht_1d_moments(
            adata, 
            covariate=cov_df,
            treatment=snps_df,
            treatment_for_gene=gene_to_snp_dictionary,
            num_boot=num_boot,
            verbose=1,
            num_cpus=num_cpu,
            resampling='bootstrap',
            approx=True,
            resample_rep=True)
        results.append(memento.get_1d_ht_result(adata))
                                                
    return pd.concat(results)


### Run memento

Due to the resampling at the single-cell level, `memento` generally takes much longer than something like Matrix eQTL or FastQTL that works on pseudobulks. It is however much faster than fitting linear mixed models for millions of cells. 

I would recommend using as many cores as you can afford on a high performance computing cluster. (for members of the Ye lab, maybe something like c5.24xlarge instance just for a few hours for this and set the num_cpu to 80). 

`memento` is fairly flexible in that if you're willing to use more CPUs, the speed will scale linearly.

In [18]:
eqtl_results = run_genetics(
    adata=adata,
    snps=snps,
    cov=cov,
    gene_snp_pairs=gene_snp_pairs,
    num_cpu=10,
    num_blocks=5, # increase this if you run out of memory.
)

blocksize 10000
working on block 0


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   11.4s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:   46.7s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:  1.8min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  3.2min
[Parallel(n_jobs=10)]: Done 947 out of 947 | elapsed:  3.9min finished


working on block 1


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    8.6s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:   46.3s
[Parallel(n_jobs=10)]: Done 408 out of 408 | elapsed:  1.7min finished


working on block 2


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    8.6s
[Parallel(n_jobs=10)]: Done 193 out of 193 | elapsed:   48.4s finished


working on block 3


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    8.8s
[Parallel(n_jobs=10)]: Done  97 out of  97 | elapsed:   26.3s finished


working on block 4


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    9.3s
[Parallel(n_jobs=10)]: Done  54 out of  54 | elapsed:   17.4s finished


In [19]:
# The de_pval is not corrected on purpose - user can correct the P-values however they please.
eqtl_results.head(10)

,gene,tx,de_coef,de_se,de_pval,dv_coef,dv_se,dv_pval
0,NOC2L,1:845635,0.005111,0.002424,0.035050,0.0,5.967713e-17,1.0
1,NOC2L,1:881627,-0.005168,0.001797,0.004051,0.0,4.361005e-17,1.0
0,SDF4,1:1062025,0.005272,0.002748,0.055024,0.0,6.480994e-17,1.0
0,UBE2J2,1:1161641,-0.000393,0.002382,0.869094,0.0,5.827615e-17,1.0
0,CCNL2,1:1252309,-0.004638,0.002607,0.075280,0.0,6.492581e-17,1.0
0,RER1,1:2229156,-0.001183,0.003967,0.765610,0.0,9.040996e-17,1.0
0,TNFRSF14,1:2560903,-0.003125,0.003016,0.300386,0.0,6.688472e-17,1.0
0,RPL22,1:6317420,0.005040,0.013121,0.701416,0.0,6.006726e-17,1.0
0,CAMTA1,1:7895069,-0.000752,0.002356,0.750012,0.0,5.653482e-17,1.0
1,CAMTA1,1:6846552,-0.003309,0.002685,0.217853,0.0,6.304167e-17,1.0
